# Exploring the API provided by wine.com

Documentation: https://api.wine.com/wiki

According to the documentation, the API contains detailed information on the wines, as well as professional ratings and community reviews. In this notebook, I just want to explore the API capabilities and the data returned.

In [105]:
import requests
import json
import pandas as pd

There are two RESTful APIs that will be helpful: the Catalog and the Category APIs. The Catalog allows you to search through all the wine data. The Category allows you to search through the categories displayed in the Catalog.

First, lets look at the Catalog. The following query returns all red wines from the wine finder, starting at record #11, with a list size of 5 products:

In [107]:
#read the API key from config.py
creds={}
with open('config.py') as file:
    exec(file.read(), creds)
garbage = creds.pop('__builtins__')

In [6]:
url_string = "http://services.wine.com/api/beta2/service.svc/json/catalog"

In [8]:
payload = {'filter': 'categories(490+124)', 'offset': '10', 'size': '5', 'apikey': creds['KEY']}

In [10]:
r= requests.get(url_string, params=payload)

In [12]:
data = json.loads(r.text)

In [14]:
data.keys()

dict_keys(['Status', 'Products'])

In [17]:
data['Products'].keys()

dict_keys(['Offset', 'Url', 'Total', 'List'])

Individual wines are stored in the 'List' key of the 'Products' dictionary. This returns a list of the wines. The documentation for the data on the wines is here: https://api.wine.com/wiki/api-object-dictionary#_catalog_objects

In [25]:
data['Products']['List'][0]

{'Appellation': {'Id': 2398,
  'Name': 'Napa Valley',
  'Region': {'Area': None,
   'Id': 101,
   'Name': 'California',
   'Url': 'http://www.wine.com/v6/California/wine/list.aspx?N=7155+101'},
  'Url': 'http://www.wine.com/v6/Napa-Valley/wine/list.aspx?N=7155+101+2398'},
 'Community': {'Reviews': {'HighestScore': 0,
   'List': [],
   'Url': 'http://www.wine.com/v6/Pahlmeyer-Napa-Valley-Proprietary-Red-2013/wine/142049/Detail.aspx?pageType=reviews'},
  'Url': 'http://www.wine.com/v6/Pahlmeyer-Napa-Valley-Proprietary-Red-2013/wine/142049/Detail.aspx'},
 'Description': '',
 'GeoLocation': {'Latitude': -360,
  'Longitude': -360,
  'Url': 'http://www.wine.com/v6/aboutwine/mapof.aspx?winery=3636'},
 'Id': 142049,
 'Labels': [{'Id': '142049m',
   'Name': 'thumbnail',
   'Url': 'http://cache.wine.com/labels/142049m.jpg'}],
 'Name': 'Pahlmeyer Napa Valley Proprietary Red 2013',
 'PriceMax': 159.99,
 'PriceMin': 155.0,
 'PriceRetail': 175.0,
 'ProductAttributes': [{'Id': 36,
   'ImageUrl': 'htt

There is a lot of data that I don't have access to at the moment, but I should be able to gain access just by asking. We'll see if that happens. In the mean time, I'll see what I can do now.

Data to extract from each 'List' element:
* ['Id'] - Id of the wine. Will be helpful for future searches.
* ['Name'] - Name of the wine.
* ['Vineyard']['Id'] - Id of the vineyard/winery.
* ['Vineyard']['Name'] - Name of the vineyard/winery.
* ['Appellation']['Id'] - Id of the appellation.
* ['Appellation']['Name'] - Name of the appellation. These are well defined geographical regions.
* ['Appellation']['Region']['Id'] - Id of the region.
* ['Appellation']['Region']['Name'] - Name of the region. It is a higher level location. For example, when the appellation is Napa Valley, the region is California.
* ['Appellation']['Region']['Area']['Id'] - Id of the area. Can be null.
* ['Appellation']['Region']['Area']['Name'] - Name of the area. This would be htings like France. Can be null. It is null for region == California.
    * The whole key called 'Area' could be null -> gives a TypeError if you try to access it.
* ['Varietal']['Id'] - Id of the varietal. Useful for searching.
* ['Varietal']['Name'] - Name of the varietal.
* ['Varietal']['Url'] - The URL for other wines of the same varietal. Can be used as a GET query.
* ['Varietal']['WineType'] - A higher level classification, e.g. red wine. Also has Id, Name, and Url.
* ['Ratings']['HighestScore'] - The highest score given by a professional rater. Only have this for now.
* ['Vintage'] - The vintage.

In [57]:
data['Products']['List'][4]['Vintage']

'2012'

For things like the Varietal Url, it is probably better to have all of the categories and just reference the Ids in the main data table. I will test out the Category API for this purpose.

In [58]:
payload_cat = {'filter': 'categories(490)', 'apikey': creds['KEY']}

In [59]:
url_string_cat = "http://services.wine.com/api/beta2/service.svc/json/categorymap"

In [60]:
r_cat = requests.get(url_string_cat, params=payload_cat)

In [61]:
data_cat = json.loads(r_cat.text)

In [63]:
data_cat.keys()

dict_keys(['Categories', 'Status'])

The Categories key holds a list.

In [69]:
data_cat['Categories'][1]['Name']

'Varietal'

It seems that the URLs are all blank in the Categories object, so I think I will just create the databases myself from the Catalog data. Scheme:
* main_wines_db: the main wine database
    * 'Wine ID': ['Id']
    * 'Wine name': ['Name']
    * 'Vintage': ['Vintage']
    * 'Score': ['Ratings']['HighestScore']
    * 'Vineyard ID': ['Vineyard']['Id']
    * 'Varietal ID': ['Varietal']['Id']
    * 'Wine Type ID': ['Varietal']['WineType']['Id']
    * 'Appellation ID': ['Appellation']['Id']
    * 'Region ID': ['Appellation']['Region']['Id']
    * 'Area ID': ['Appellation']['Region']['Area']['Id']
    
* varietals_db: the database of varietals
    * 'Varietal ID': ['Varietal']['Id']
    * 'Varietal Name': ['Varietal']['Name']
    * 'Varietal URL': ['Varietal']['Url']
    
* wine_types_db: the database of wine types
    * 'Wine Type ID': ['WineType']['Id']
    * 'Wine Type Name': ['WineType']['Name']
    * 'Wine Type URL': ['WineType']['Url']

* appellations_db: the database of regions
    * 'Appellation ID': ['Appellation']['Id']
    * 'Appellation Name': ['Appellation']['Name']
    
* regions_db: the database of regions
    * 'Region ID': ['Appellation']['Region']['Id']
    * 'Region Name': ['Appellation']['Region']['Name']
    
* areas_db: the database of areas
    * 'Area ID': ['Appellation']['Region']['Area']['Id']
    * 'Area Name': ['Appellation']['Region']['Area']['Name']
    
* vineyards_db: the database of vineyards/wineries
    * 'Vineyard ID': ['Vineyard']['Id']
    * 'Vineyard Name': ['Vineyard']['Name']

In [198]:
payload['size'] = 100

In [199]:
r2 = requests.get(url_string, params=payload)

In [201]:
data2 = json.loads(r2.text)

In [233]:
data2['Status']['ReturnCode']

0

Since I might be able to get access to Category data later, I won't handle duplicates yet.

In [203]:
main_wines = []
varietals = []
wine_types = []
appellations = []
regions = []
locations = []
areas = []

for item in data2['Products']['List']:
    #get the data for main_wine
    wine = [item['Id'],
            item['Name'],
            item['Vintage'],
            item['Ratings']['HighestScore'],
            item['Vineyard']['Id'],
            item['Varietal']['Id'],
            item['Varietal']['WineType']['Id'],
            item['Appellation']['Id'],
            item['Appellation']['Region']['Id']]
    
    #handle area since it can be None
    try:
        area = item['Appellation']['Region']['Area']['Id']
    except TypeError:
        area = None
    
    #add area
    wine.append(area)
    
    #append to main_wine
    main_wines.append(wine)
    
    #get the data for varietals
    varietal = [item['Varietal']['Id'],
                 item['Varietal']['Name'],
                 item['Varietal']['Url']]
    
    #append to varietals
    varietals.append(varietal)
    
    #get the data for wine_types
    wine_type = [item['Varietal']['WineType']['Id'],
                 item['Varietal']['WineType']['Name'],
                 item['Varietal']['WineType']['Url']]
    
    #append to wine_types
    wine_types.append(wine_type)
    
    #get the data for appellations
    appellation = [item['Appellation']['Id'],
                item['Appellation']['Name']]
    
    #append to appellations
    appellations.append(appellation)
    
    #get the data for regions
    region = [item['Appellation']['Region']['Id'],
              item['Appellation']['Region']['Name']]
    
    #append to regions
    regions.append(region)
    
    #try to get data for areas
    try:
        area = [item['Appellation']['Region']['Area']['Id'],
                item['Appellation']['Region']['Area']['Name']]
    except TypeError:
        pass
    else:
        areas.append(area) #only do this if try works
    
    #get the data for vineyards
    vineyard = [item['Vineyard']['Id'], item['Vineyard']['Name']]
    
    #append to vineyards
    vineyards.append(vineyard)

In [204]:
main_wines

[[142049,
  'Pahlmeyer Napa Valley Proprietary Red 2013',
  '2013',
  96,
  7270,
  144,
  124,
  2398,
  101,
  None],
 [153157,
  'Silver Oak Napa Valley Cabernet Sauvignon 2011',
  '2011',
  91,
  7396,
  139,
  124,
  2398,
  101,
  None],
 [154691,
  'Haras de Pirque Hussonet Gran Reserva Cabernet Sauvignon 2011',
  '2011',
  95,
  6024,
  139,
  124,
  2428,
  112,
  None],
 [155466,
  'Joseph Phelps Insignia 2013',
  '2013',
  100,
  8881,
  144,
  124,
  2398,
  101,
  None],
 [158048,
  'The Debate Three Vineyard Collection 3-pack 2012',
  '2012',
  100,
  999997220,
  139,
  124,
  2398,
  101,
  None],
 [162035, 'Verite La Joie 2013', '2013', 100, 8509, 144, 124, 2371, 101, None],
 [103730,
  'Chateau Lafite Rothschild  2008',
  '2008',
  98,
  999999160,
  144,
  124,
  10049,
  10038,
  None],
 [154233,
  'Caymus Special Selection Cabernet Sauvignon 2013',
  '2013',
  95,
  8626,
  139,
  124,
  2398,
  101,
  None],
 [143695,
  'Moss Roxx Ancient Vine Zinfandel 2013',
  '

In [205]:
len(main_wines)

1000

In [206]:
varietals

[[144,
  'Bordeaux Red Blends',
  'http://www.wine.com/v6/Bordeaux-Red-Blends/wine/list.aspx?N=7155+124+144'],
 [139,
  'Cabernet Sauvignon',
  'http://www.wine.com/v6/Cabernet-Sauvignon/wine/list.aspx?N=7155+124+139'],
 [139,
  'Cabernet Sauvignon',
  'http://www.wine.com/v6/Cabernet-Sauvignon/wine/list.aspx?N=7155+124+139'],
 [144,
  'Bordeaux Red Blends',
  'http://www.wine.com/v6/Bordeaux-Red-Blends/wine/list.aspx?N=7155+124+144'],
 [139,
  'Cabernet Sauvignon',
  'http://www.wine.com/v6/Cabernet-Sauvignon/wine/list.aspx?N=7155+124+139'],
 [144,
  'Bordeaux Red Blends',
  'http://www.wine.com/v6/Bordeaux-Red-Blends/wine/list.aspx?N=7155+124+144'],
 [144,
  'Bordeaux Red Blends',
  'http://www.wine.com/v6/Bordeaux-Red-Blends/wine/list.aspx?N=7155+124+144'],
 [139,
  'Cabernet Sauvignon',
  'http://www.wine.com/v6/Cabernet-Sauvignon/wine/list.aspx?N=7155+124+139'],
 [141,
  'Zinfandel',
  'http://www.wine.com/v6/Zinfandel/wine/list.aspx?N=7155+124+141'],
 [139,
  'Cabernet Sauvignon'

In [207]:
wine_types

[[124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.com/v6/Red-Wines/wine/list.aspx?N=7155+124'],
 [124,
  'Red Wines',
  'http://www.wine.co

In [208]:
appellations

[[2398, 'Napa Valley'],
 [2398, 'Napa Valley'],
 [2428, 'Chile'],
 [2398, 'Napa Valley'],
 [2398, 'Napa Valley'],
 [2371, 'Sonoma County'],
 [10049, 'Pauillac'],
 [2398, 'Napa Valley'],
 [2388, 'Other California'],
 [2398, 'Napa Valley'],
 [2398, 'Napa Valley'],
 [10049, 'Pauillac'],
 [2371, 'Sonoma County'],
 [2452, 'Tuscany'],
 [2452, 'Tuscany'],
 [2398, 'Napa Valley'],
 [10049, 'Pauillac'],
 [2370, 'Rioja'],
 [10049, 'Pauillac'],
 [2474, 'Willamette Valley'],
 [10043, 'Pomerol'],
 [2452, 'Tuscany'],
 [2052, 'Yakima Valley'],
 [2398, 'Napa Valley'],
 [2371, 'Sonoma County'],
 [2398, 'Napa Valley'],
 [9063, 'McLaren Vale'],
 [2398, 'Napa Valley'],
 [10044, 'St. Estephe'],
 [2462, 'Central Coast'],
 [2398, 'Napa Valley'],
 [2371, 'Sonoma County'],
 [10066, 'Sicily'],
 [10043, 'Pomerol'],
 [2452, 'Tuscany'],
 [10043, 'Pomerol'],
 [2371, 'Sonoma County'],
 [10049, 'Pauillac'],
 [10044, 'St. Estephe'],
 [10049, 'Pauillac'],
 [2452, 'Tuscany'],
 [2398, 'Napa Valley'],
 [2398, 'Napa Valley'

In [209]:
regions

[[101, 'California'],
 [101, 'California'],
 [112, 'South America'],
 [101, 'California'],
 [101, 'California'],
 [101, 'California'],
 [10038, 'France - Bordeaux'],
 [101, 'California'],
 [101, 'California'],
 [101, 'California'],
 [101, 'California'],
 [10038, 'France - Bordeaux'],
 [101, 'California'],
 [105, 'Italy'],
 [105, 'Italy'],
 [101, 'California'],
 [10038, 'France - Bordeaux'],
 [109, 'Spain'],
 [10038, 'France - Bordeaux'],
 [103, 'Oregon'],
 [10038, 'France - Bordeaux'],
 [105, 'Italy'],
 [104, 'Washington'],
 [101, 'California'],
 [101, 'California'],
 [101, 'California'],
 [108, 'Australia'],
 [101, 'California'],
 [10038, 'France - Bordeaux'],
 [101, 'California'],
 [101, 'California'],
 [101, 'California'],
 [105, 'Italy'],
 [10038, 'France - Bordeaux'],
 [105, 'Italy'],
 [10038, 'France - Bordeaux'],
 [101, 'California'],
 [10038, 'France - Bordeaux'],
 [10038, 'France - Bordeaux'],
 [10038, 'France - Bordeaux'],
 [105, 'Italy'],
 [101, 'California'],
 [101, 'Califo

In [210]:
areas

[]

In [211]:
vineyards

[[7270, 'Pahlmeyer'],
 [7396, 'Silver Oak Napa Valley'],
 [6024, 'Haras de Pirque'],
 [8881, 'Joseph Phelps Vineyards'],
 [999997220, 'The Debate'],
 [8509, 'Verite Estate'],
 [8626, 'Caymus'],
 [999997365, 'Moss Roxx'],
 [8626, 'Caymus'],
 [8808, 'Duckhorn Vineyards'],
 [999999028, 'Château Latour'],
 [999998522, 'Silver Oak Alexander Valley'],
 [5616, 'Tenuta Sette Ponti'],
 [7746, 'Fattoria dei Barbi'],
 [8798, 'Freemark Abbey'],
 [999999692, 'Carruades de Lafite'],
 [8220, 'La Rioja Alta'],
 [999999127, 'Chateau Duhart-Milon'],
 [8693, 'Beaux Freres'],
 [999999170, 'Chateau Petrus'],
 [7270, 'Pahlmeyer'],
 [7396, 'Silver Oak Napa Valley'],
 [6024, 'Haras de Pirque'],
 [8881, 'Joseph Phelps Vineyards'],
 [999997220, 'The Debate'],
 [8509, 'Verite Estate'],
 [8626, 'Caymus'],
 [999997365, 'Moss Roxx'],
 [8626, 'Caymus'],
 [8808, 'Duckhorn Vineyards'],
 [999999028, 'Château Latour'],
 [999998522, 'Silver Oak Alexander Valley'],
 [5616, 'Tenuta Sette Ponti'],
 [7746, 'Fattoria dei Barb

Just a quick note: The order appears to be the same, so I can use offset to systematically get the data.
Another note: I should check area once I'm done. I see a lot of None's.

Now to turn these into DataFrames. Here is where I'll remove duplicates.

In [212]:
main_wines_cols = ['Wine ID',
                   'Wine name',
                   'Vintage',
                   'Score',
                   'Vineyard ID',
                   'Varietal ID',
                   'Wine Type ID',
                   'Appellation ID',
                   'Region ID',
                   'Area ID']

In [213]:
main_wines_db = pd.DataFrame(main_wines, columns=main_wines_cols)
main_wines_db = main_wines_db.drop_duplicates()

In [214]:
main_wines_db.head()

,Wine ID,Wine name,Vintage,Score,Vineyard ID,Varietal ID,Wine Type ID,Appellation ID,Region ID,Area ID
0,142049,Pahlmeyer Napa Valley Proprietary Red 2013,2013,96,7270,144,124,2398,101,None
1,153157,Silver Oak Napa Valley Cabernet Sauvignon 2011,2011,91,7396,139,124,2398,101,None
2,154691,Haras de Pirque Hussonet Gran Reserva Cabernet...,2011,95,6024,139,124,2428,112,None
3,155466,Joseph Phelps Insignia 2013,2013,100,8881,144,124,2398,101,None
4,158048,The Debate Three Vineyard Collection 3-pack 2012,2012,100,999997220,139,124,2398,101,None


In [215]:
varietals_cols = ['Varietal ID', 'Varietal Name', 'Varietal URL']

In [216]:
varietals_db = pd.DataFrame(varietals, columns=varietals_cols)
varietals_db = varietals_db.drop_duplicates()

In [217]:
varietals_db

,Varietal ID,Varietal Name,Varietal URL
0,144,Bordeaux Red Blends,http://www.wine.com/v6/Bordeaux-Red-Blends/win...
1,139,Cabernet Sauvignon,http://www.wine.com/v6/Cabernet-Sauvignon/wine...
8,141,Zinfandel,http://www.wine.com/v6/Zinfandel/wine/list.asp...
13,145,Other Red Blends,http://www.wine.com/v6/Other-Red-Blends/wine/l...
14,163,Sangiovese,http://www.wine.com/v6/Sangiovese/wine/list.as...
17,169,Tempranillo,http://www.wine.com/v6/Tempranillo/wine/list.a...
19,143,Pinot Noir,http://www.wine.com/v6/Pinot-Noir/wine/list.as...
24,138,Merlot,http://www.wine.com/v6/Merlot/wine/list.aspx?N...
26,146,Syrah/Shiraz,http://www.wine.com/v6/SyrahShiraz/wine/list.a...
68,10082,Rhone Red Blends,http://www.wine.com/v6/Rhone-Red-Blends/wine/l...


In [218]:
wine_types_cols = ['Wine Type ID', 'Wine Type Name', 'Wine Type URL']

In [219]:
wine_types_db = pd.DataFrame(wine_types, columns=wine_types_cols)
wine_types_db = wine_types_db.drop_duplicates()

In [220]:
wine_types_db

,Wine Type ID,Wine Type Name,Wine Type URL
0,124,Red Wines,http://www.wine.com/v6/Red-Wines/wine/list.asp...


Remember that red wines was in the search query.

In [221]:
appellations_cols = ['Appellation ID', 'Appellation Name']

In [222]:
appellations_db = pd.DataFrame(appellations, columns=appellations_cols)
appellations_db = appellations_db.drop_duplicates()

In [223]:
appellations_db.head()

,Appellation ID,Appellation Name
0,2398,Napa Valley
2,2428,Chile
5,2371,Sonoma County
6,10049,Pauillac
8,2388,Other California


In [224]:
regions_cols = ['Region ID', 'Region Name']

In [225]:
regions_db = pd.DataFrame(regions, columns=regions_cols)
regions_db = regions_db.drop_duplicates()

In [226]:
regions_db

,Region ID,Region Name
0,101,California
2,112,South America
6,10038,France - Bordeaux
13,105,Italy
17,109,Spain
19,103,Oregon
22,104,Washington
26,108,Australia
50,114,New Zealand
68,10039,France - Rhone


In [227]:
areas_cols = ['Areas ID', 'Areas Name']

In [228]:
areas_db = pd.DataFrame(areas, columns=areas_cols)
areas_db = areas_db.drop_duplicates()

In [229]:
areas_db

,Areas ID,Areas Name


So, currently, there is no area data.

In [230]:
vineyards_cols = ['Vineyard ID', 'Vineyard Name']

In [231]:
vineyards_db = pd.DataFrame(vineyards, columns=vineyards_cols)
vineyards_db = vineyards_db.drop_duplicates()

In [232]:
vineyards_db.head()

,Vineyard ID,Vineyard Name
0,7270,Pahlmeyer
1,7396,Silver Oak Napa Valley
2,6024,Haras de Pirque
3,8881,Joseph Phelps Vineyards
4,999997220,The Debate


I'm going to forget about the area code. It seems empty most of the time, so I'll forget about it.

In [234]:
payload_test = {'filter': 'product(153157)', 'apikey': creds['KEY']}

In [235]:
r_prod = requests.get(url_string, params=payload_test)

In [236]:
data_prod = json.loads(r_prod.text)

In [241]:
data_prod['Products']['List'][0]['Community']['Reviews']['Url']

'http://www.wine.com/v6/Silver-Oak-Napa-Valley-Cabernet-Sauvignon-2011/wine/153157/Detail.aspx?pageType=reviews'